In [1]:
import pandas as pd
import string,re
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional,Permute,Reshape,Multiply,Flatten,Activation,RepeatVector

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
data=pd.read_csv('/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv')
data['question'] = data['question'].str.lower().apply(lambda x: re.sub(r'[^\w\s\']', '', x))
data['answer'] = data['answer'].str.lower().apply(lambda x: re.sub(r'[^\w\s\']', '', x))
new_train = data['question']
new_target = data['answer']

In [3]:
import pandas as pd
data = pd.concat([new_train, new_target], axis=0)
data = data.reset_index(drop=True)
print(len(data))

7450


In [4]:
train,target=[],[]
count=0
for i in range(len(data)):
    string =""
    listt=data[i].split()
    for j in range(len(listt)-1):
        string+=listt[j]
        string+=" "
        train.append(string)
        target.append(listt[j+1])

In [5]:
index_token={}
index_detoken={}
data_vocab=set()
for i in data:
    for j in i.split():
        data_vocab.add(j)
data_vocab=list(data_vocab)
for i in range(1,len(data_vocab)+1):
    index_token[data_vocab[i-1]]=i
for i in range(1,len(data_vocab)+1):
    index_detoken[i]=data_vocab[i-1]
print("hi")
count1=0
count2=0
for keys,value in index_token.items():
    print(keys,value)
    count1+=1
    if(count1==5): break
for keys,value in index_detoken.items():
    print(keys,value)
    count2+=1
    if(count2==5): break

hi
bush 1
you'll 2
comes 3
cleaner 4
not 5
1 bush
2 you'll
3 comes
4 cleaner
5 not


In [6]:
for i in range(5):
    print(train[i],target[i])

hi  how
hi how  are
hi how are  you
hi how are you  doing
i'm  fine


In [7]:
train_token_data=[]
target_token_data=[]
for i in train:
    listt1=[]
    for j in i.split():
        listt1.append(index_token[j])
    train_token_data.append(listt1)

for i in target:
    listt1=[]
    for j in i.split():
        listt1.append(index_token[j])
    target_token_data.append(listt1)

In [8]:
for i in range(10):
    print(train[i],target[i])
    print(train_token_data[i],target_token_data[i])

hi  how
[872] [1339]
hi how  are
[872, 1339] [813]
hi how are  you
[872, 1339, 813] [675]
hi how are you  doing
[872, 1339, 813, 675] [2358]
i'm  fine
[867] [89]
i'm fine  how
[867, 89] [1339]
i'm fine how  about
[867, 89, 1339] [516]
i'm fine how about  yourself
[867, 89, 1339, 516] [132]
i'm  pretty
[867] [1837]
i'm pretty  good
[867, 1837] [2500]


In [9]:
max_seq_len=max([len(seq) for seq in train_token_data])
print(max_seq_len)

18


In [10]:
# Convert the tokenized sequences to numpy arrays
inputt = np.array(train_token_data)
targett = np.array(target_token_data)

/tmp/ipykernel_28/3705883231.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  inputt = np.array(train_token_data)


In [11]:
from tensorflow.keras.utils import to_categorical
target_words = targett.flatten()
num_classes = len(index_token)+1
target_words = to_categorical(target_words, num_classes=num_classes)

In [12]:
inputt = pad_sequences(inputt, maxlen=max_seq_len, padding='post')

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(inputt, target_words, test_size=0.2)

In [14]:
embedding_size = 100
inputs = Input(shape=(max_seq_len,))
x = Embedding(input_dim=num_classes, output_dim=embedding_size)(inputs)
# write return_sequences=True if you are adding lstm layer because lstm wants 3D tensor shape which is [batch_size,timesetep,units].
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x=LSTM(256)(x)
x=Flatten()(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 18)]              0         
                                                                 
 embedding (Embedding)       (None, 18, 100)           253300    
                                                                 
 bidirectional (Bidirectiona  (None, 18, 256)          234496    
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 2533)              650981    
                                                             

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)

Epoch 1/50
1013/1013 [==============================] - 33s 23ms/step - loss: 6.3516 - accuracy: 0.0371 - val_loss: 6.3096 - val_accuracy: 0.0338
Epoch 2/50
1013/1013 [==============================] - 13s 13ms/step - loss: 6.2071 - accuracy: 0.0355 - val_loss: 6.3111 - val_accuracy: 0.0368
Epoch 3/50
1013/1013 [==============================] - 12s 12ms/step - loss: 6.1266 - accuracy: 0.0419 - val_loss: 6.1150 - val_accuracy: 0.0543
Epoch 4/50
1013/1013 [==============================] - 12s 12ms/step - loss: 5.8220 - accuracy: 0.0572 - val_loss: 5.7806 - val_accuracy: 0.0647
Epoch 5/50
1013/1013 [==============================] - 12s 12ms/step - loss: 5.4418 - accuracy: 0.0846 - val_loss: 5.5180 - val_accuracy: 0.0978
Epoch 6/50
1013/1013 [==============================] - 12s 11ms/step - loss: 5.1357 - accuracy: 0.1099 - val_loss: 5.2848 - val_accuracy: 0.1133
Epoch 7/50
1013/1013 [==============================] - 12s 12ms/step - loss: 4.8603 - accuracy: 0.1294 - val_loss: 5.0968 -

In [18]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test accuracy: {accuracy}')

254/254 [==============================] - 2s 6ms/step - loss: 3.7732 - accuracy: 0.5516
Test accuracy: 0.5516049265861511


In [19]:
def predict_next_word(model, input_text, max_seq_len):
    # Preprocess the input text
    input_text = input_text.lower()
    input_text = re.sub(r'[^\w\s\']', '', input_text)
    input_tokens=[]
    for i in input_text.split():
        input_tokens.append(index_token[i])
    input_tokens = pad_sequences([input_tokens], maxlen=max_seq_len, padding='post')
    # Generate the prediction
    prediction = model.predict(input_tokens)[0]
    # Find the index of the most likely next word
    next_word_idx = np.argmax(prediction)
    # Convert the index to a word
    next_word=index_detoken[next_word_idx]
    
    return next_word

sent=["it's not bad there are a",
      "hello",
      "i'm",
      "what about",
      "you",
      "good luck",
      "would you like to see a",
     "thank you for",
      "I'm good and",
      "what's going?",
     "It was good, I just stayed home and",
      " It was great, thanks for "
     ]
for input_text in sent:
    next_word = predict_next_word(model, input_text, max_seq_len)
    print(input_text)
    print(f'Next word: {next_word}')


1/1 [==============================] - 1s 1s/step
it's not bad there are a
Next word: lot
1/1 [==============================] - 0s 19ms/step
hello
Next word: may
1/1 [==============================] - 0s 19ms/step
i'm
Next word: not
1/1 [==============================] - 0s 19ms/step
what about
Next word: the
1/1 [==============================] - 0s 19ms/step
you
Next word: should
1/1 [==============================] - 0s 20ms/step
good luck
Next word: with
1/1 [==============================] - 0s 20ms/step
would you like to see a
Next word: movie
1/1 [==============================] - 0s 19ms/step
thank you for
Next word: inviting
1/1 [==============================] - 0s 19ms/step
I'm good and
Next word: yet
1/1 [==============================] - 0s 21ms/step
what's going?
Next word: on
1/1 [==============================] - 0s 25ms/step
It was good, I just stayed home and
Next word: then
1/1 [==============================] - 0s 19ms/step
 It was great, thanks for 
Next word: 30
